<a href="https://colab.research.google.com/github/NikolayLenkovNikolaev/SAS-in-Clinical-Trial/blob/main/B_L6a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression models for correlated data.
# Linear models for longitudinal data

## Previous example
Aim: To study the satisfaction of patients with their medical doctors’ treatments.

fig-13

- $\sigma_e^2=1.0308$

- $\sigma_u^2 = 0.8422$

- $ICC = \frac{0.8422}{1.0308+0.8422} = 0.45=\rho$

```
PROC MIXED DATA=MD;
CLASS MD;
MODEL Y= / SOLUTION;
RANDOM INTERCEPT / SUBJECT=MD;
QUIT
```


level 1: patient
- $y_{ij} = \alpha_i + e_{ij}$
- $e_{ij} \sim N(0, \sigma_e^2)$

Level 2: MD:
- $\alpha_i = \alpha +u_i$
- $u_i  \sim N(0, \sigma_u^2)$

- i=cluster
- j-subject


## Compound symmetry covariance matrix
Variance-covariance matrix: compound symmetry (CS)


- fig 14

```
PROC MIXED DATA=MD;
CLASS MD;
MODEL Y= / SOLUTION;
REPEATED / SUBJECT=MD TYPE=CS;
QUIT;
```

```
PROC MIXED DATA=MD;
CLASS MD;
MODEL Y= / SOLUTION;
REPEATED / SUBJECT=MD TYPE=CS R RCORR;
QUIT;
```
R: covariance matrix

RCORR: correlation ICC matrix

fig-15





## Longitudinal data

- multiple observations are collected from a single subject (the cluster is
the subject himself) (longitudinal or repeated measures data)

## Longitudinal Data / Repeated Measures

```
PROC SGPANEL DATA=T1;
PANELBY TREAT PATIENT;
SCATTER Y=Y X=TIME;
RUN;

```

- fig 16


```
2
3
Independence Model
Longitudinal Data / Repeated Measures
PROC REG DATA=T1;
MODEL Y=TREAT;
QUIT;
```


TREATMENT IS A CLUSTER CONSTANT COVARIATE

- fig 17

```
PROC MIXED DATA=T1;
CLASS PATIENT;
MODEL Y=TREAT / SOLUTION;
RANDOM PATIENT;
RUN;
```


- $\sigma_e^2= 0.1087$

- $\sigma_u^2 = 0.7847$

- $ICC = \frac{0.7847}{0.1087+0.7847} = 0.88=\rho$

```
PROC MIXED DATA=T1;
CLASS PATIENT;
MODEL Y=TREAT / SOLUTION;
REPEATED / SUBJECT=PATIENT
TYPE=CS R RCORR;
RUN;
```

There are several ways a mixed model can be used to analyse repeated
measures data.

The simplest approach is to use a model with patient effects fitted as
random (random intercepts model). This will allow for a constant
correlation between all observations on the same patient.

Exchangeable or Compound Symmetry (CS) Correlation Structure
Every observation within an individual is equally correlated with every other
observation in that individual.

fig-18

Often the correlation between observations on the same patient is not
constant. For example, correlation may decrease as visits become more
widely separated in time.

First order Auto-Regressive - AR(1) - Correlation Structure
Two observations taken close in time (or space) within an individual tend to be
more correlated than two observations take far apart in the same individual

- fig 18 -left

## Which covariance pattern to consider?

There are many covariance patterns available and choosing the most
appropriate is not always easy.

Mixed Models with different covariance patterns can be compared
statistically using Likelihood Ratio Tests provided that they fit the same
fixed effects and their covariance patterns are nested (e.g. a compound
symmetry patterns is nested within a general structure pattern, but it is not
nested with a first-order autoregressive pattern).
If the covariance parameters in the models compared are not nested,
Akaike’s criteria (AIC) could be used.

```
PROC MIXED DATA=AR1;
CLASS PATIENT;
MODEL Y=/ SOLUTION;
REPEATED / SUBJECT=PATIENT TYPE=CS RCORR;
RUN;
PROC MIXED DATA=AR1;
CLASS PATIENT;
MODEL Y=/ SOLUTION;
REPEATED / SUBJECT=PATIENT TYPE=AR(1) RCORR;
RUN;

PROC SGPANEL DATA=AR1;
PANELBY PATIENT;
SCATTER Y=Y X=TIME;
RUN;
```

fig 19

## Within-Subject Varying Covariates

Now we want estimate the effect of time on the outcome.

Repeated measures are taken at different time points on the same subject.

In this situation, the effect of interest (TIME) varies within subject.

If we have only two measures per subject (e.g., before and after an
intervention), we know that using a t-test for independent sample (that is
ignoring correlation within subject) would lead us a variance estimate of the
effect of interest that is too large (a test based on high-variance estimate will result in accepting the null hypothesis of no time effect too often).

The appropriate analysis here is a t test for paired data.

```
DATA LONG(KEEP=SUBJ Y TIME);
SET PESO;
Y=PRIMA;TIME=0;OUTPUT;
Y=DOPO;TIME=1;OUTPUT;
RUN;
PROC PRINT DATA=LONG;RUN;

```

Two independent sample t-test:
- Diff=-3.46 s.e.: 8.53 p-value: 0.6866

Paired data t-test
- Diff=-3.46 s.e.: 1.07 p-value: 0.0028

```
PROC MIXED DATA=LONG;
CLASS SUBJ;
MODEL Y=TIME / SOLUTION;
RANDOM INTERCEPT / SUBJECT=SUBJ;
RUN;
```


The use of regression models for correlated data allow us to extend
the previous analysis to more than two repeated measures on the
same subject.




## Within-Subject Varying Covariates

Estimate of Time effect: Independence Model

$y_{ij}= \alpha + \beta.TIME_{ij} +e_{ij}$

$\sigma_e^2 = 2.18$

```
PROC MIXED DATA=AR1;
CLASS PATIENT;
MODEL Y=TIME / SOLUTION;
RANDOM INTERCEPT / SUBJECT=PATIENT SOLUTION;
RUN;

PROC MIXED DATA=AR1;
CLASS PATIENT;
MODEL Y=TIME / SOLUTION;
RANDOM INTERCEPT / SUBJECT=PATIENT;
RUN;
/*SAME MODEL*/
PROC MIXED DATA=AR1;
CLASS PATIENT;
MODEL Y=TIME / SOLUTION;
REPEATED / SUBJECT=PATIENT TYPE=CS RCORR;
RUN;

PROC MIXED DATA=AR1;
CLASS PATIENT;
MODEL Y=TIME / SOLUTION;
REPEATED / SUBJECT=PATIENT TYPE=AR(1) RCORR;
RUN;
```


## Within- and Between- Subject Varying Covariates

Estimate of Time´Treatment effect: Independence Mode

$y_{ij} = \alpha +\beta.TIME_{ij} +\gamma.Y_{ij}+\delta.TIME_{ij}.T_{ij} +e_{ij}$

```
PROC MIXED DATA=TIMETREAT;
CLASS TREAT;
MODEL Y=TIME TREAT TIME*TREAT / SOLUTION;
RUN;
```


## Other issues in the analysis of correlated data
- Unbalanced data and missing data
- Estimation methods (ML, REML, PQL, etc.)
- More than two levels in the data hierarchy
- binary outcomes and count data – (Generalized Linear Mixed Models –
PROC GLIMMIX in SAS)
- Bayesian approach
- Software
- Generalized Estimating Equation models (GEE)
- Sample size estimation for clustered intervention studies
- …

